This notebook trains and evaluates a prototype D-MPNN on QM9 (regression)

The code is based on run_training.py

To investigate:

- Why do only two learned weight matrices have a bias switch?
- Loss function... why are we dividing by number of molecules twice?

In [1]:
import csv
import os
import sys
import numpy as np
import torch
import pickle
import copy

from logging import Logger
from typing import List
from tqdm import trange

from torch.optim.lr_scheduler import ExponentialLR
from torch_geometric.datasets import QM9

In [2]:
# cd to chempropBayes
%cd /Users/georgelamb/Documents/GitHub/chempropBayes

/Users/georgelamb/Documents/GitHub/chempropBayes


In [3]:
# import from chempropBayes
from chemprop.train.evaluate import evaluate, evaluate_predictions
from chemprop.train.predict import predict
from chemprop.train.train import train
from chemprop.args import TrainArgs
from chemprop.data import StandardScaler, MoleculeDataLoader
from chemprop.data.utils import get_class_sizes, get_data, get_task_names, split_data
from chemprop.models import MoleculeModel
from chemprop.nn_utils import param_count
from chemprop.utils import build_optimizer, build_lr_scheduler, get_loss_func, get_metric_func, load_checkpoint,\
    makedirs, save_checkpoint, save_smiles_splits

In [4]:
# instantiate args class and load from dict
args = TrainArgs()
args.from_dict({
    'dataset_type': 'regression',
    'data_path': '/Users/georgelamb/Documents/GitHub/chempropBayes/data/QM9.csv'
})

# define logger or writer
logger = None
writer = None

### 1) Prepare data

Get the data

In [5]:
# ARGS
args.target_columns = ['mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'cv', 'u0', 'u298', 'h298', 'g298']
args.task_names = args.target_columns
args.features_path = None # path to load additional features
args.features_generator = None # generator for additional features (cannot specify generator and path)
args.max_data_size = 20000 # number of data points to load

In [6]:
# use get_data function to convert QM9 csv -> MoleculeDataset
# a MoleculeDataset contains a list of molecules and their associated features and targets
data = get_data(path=args.data_path, args=args)

# call MoleculeDataset methods
args.num_tasks = data.num_tasks() # returns number of prediction tasks (number of targets)
args.features_size = data.features_size() # returns size of features array for each molecule (>0 or None)


14356it [00:00, 52034.67it/s] 
100%|██████████| 20000/20000 [00:01<00:00, 15939.00it/s]


Split the data

In [7]:
# ARGS
# split type options: 'random', 'scaffold_balanced', 'predetermined', 'crossval', 'index_predetermined'
args.split_type = 'random' # specify split type 
args.split_sizes = (0.8, 0.1, 0.1) # tuple of train/val/test proportions
args.seed = 0 # random seed for splitting

In [8]:
# split_data returns three MoleculeDatasets
train_data, val_data, test_data = split_data(data=data, split_type=args.split_type, 
                                             sizes=args.split_sizes, seed=args.seed, args=args, logger=logger)
# specify training data size
args.train_data_size = len(train_data)


Standardise

In [9]:
# standardise training targets (zero mean, unit variance)
if args.dataset_type == 'regression':
    train_smiles, train_targets = train_data.smiles(), train_data.targets() # return training targets
    scaler = StandardScaler().fit(train_targets) # fit transform to training targets
    scaled_targets = scaler.transform(train_targets).tolist() # apply transform
    train_data.set_targets(scaled_targets) # replace targets in MoleculeDataset with standardised targets


Create data loaders

In [10]:
# ARGS
args.cache_cutoff = 10000 # max number of molecules for caching
args.batch_size = 50 # default batch size is 50
args.num_workers = 8 # number of workers for parallel data loading
args.class_balance = False # equal number of pos and neg in each batch (for classification)
args.seed = 0 # seed for data loader shuffle

In [11]:
# determine whether to cache graph featurisations when running batch_graph method
# caching = saving mol_graphs in SMILES_TO_GRAPH
if len(data) <= args.cache_cutoff:
    cache = True
    num_workers = 0
else: # if we don't cache we perform parallel data loading
    cache = False
    num_workers = args.num_workers
    
# instantiate data loaders
# MoleculeDataLoader: an iterable over a MoleculeDataSet which calls MoleculeSampler
# data loader returns a MoleculeDataSet with BatchMolGraph computed using batch_graph method
train_data_loader = MoleculeDataLoader(
        dataset=train_data,
        batch_size=args.batch_size,
        num_workers=num_workers,
        cache=cache,
        class_balance=args.class_balance,
        shuffle=True,
        seed=args.seed
)
val_data_loader = MoleculeDataLoader(
        dataset=val_data,
        batch_size=args.batch_size,
        num_workers=num_workers,
        cache=cache
)
test_data_loader = MoleculeDataLoader(
        dataset=test_data,
        batch_size=args.batch_size,
        num_workers=num_workers,
        cache=cache
)


### 2) Build model

Define loss and metric functions

In [12]:
# ARGS
args.metric = 'mae' # set metric

In [13]:
loss_func = get_loss_func(args) # returns loss function based on args.dataset_type (MSE for regression)
metric_func = get_metric_func(metric=args.metric) # returns callable metric function based on args.metric


Instantiate model

In [14]:
# ARGS: seed
args.pytorch_seed = 0

# ARGS: additional features
args.features_only = False # if True only the 'additional' features feed the FFN

# ARGS: message passing
args.atom_messages = False # False means bond-centred i.e. D-MPNN
args.undirected = False # if messages are bond-centred we can choose directed or undirected
args.hidden_size = 300 # hidden layer size
args.depth = 3 # number of message passing steps
args.bias = False # only affects W_i and W_h (only message passing layers where you turn it off)
args.device = torch.device('cpu')

# ARGS: FFN
args.ffn_hidden_size = 300
args.ffn_num_layers = 2

# ARGS: both message passing and FFN
args.dropout = 0.0 # dropout probability
args.activation = 'ReLU'


In [15]:
# set pytorch seed for random initial weights
torch.manual_seed(args.pytorch_seed);

# MoleculeModel: message passing network followed by feed-forward layers
model = MoleculeModel(args)


Build optimiser and lr scheduler

In [16]:
# ARGS

# lr scheduler (Noam)
# linear increase from init_lr to max_lr during warmup_steps (warmup_epochs * steps_per_epoch)
# steps_per_epoch = train_data_size // batch_size
# exponential decay from max_lr to final_lr over remaining steps
# total epochs = args.epochs * args.num_lrs

args.init_lr = 1e-4
args.max_lr = 1e-3
args.final_lr = 1e-4

args.warmup_epochs = 2.0
args.epochs = 30

In [17]:
# build optimiser
optimizer = build_optimizer(model, args)

# build scheduler
scheduler = build_lr_scheduler(optimizer, args)


### 3) Train and test

Train and save model with best val score

In [18]:
# ARGS
args.log_frequency = 320 # number of batches between each logging of the training loss
args.show_individual_scores = False

In [24]:
# training loop
best_score = float('inf') if args.minimize_score else -float('inf')
best_epoch, n_iter = 0, 0
for epoch in range(args.epochs):
    
    # train for one epoch
    n_iter = train(
        model=model,
        data_loader=train_data_loader,
        loss_func=loss_func,
        optimizer=optimizer,
        scheduler=scheduler,
        args=args,
        n_iter=n_iter,
        logger=logger,
        writer=writer
    )
    
    # evaluation on val set for one epoch
    val_scores = evaluate(
        model=model,
        data_loader=val_data_loader,
        num_tasks=args.num_tasks,
        metric_func=metric_func,
        dataset_type=args.dataset_type,
        scaler=scaler,
        logger=logger
    )
    
    # average validation score
    avg_val_score = np.nanmean(val_scores)
    print(f'Validation AVG {args.metric} = {avg_val_score:.6f}')
    
    # show individual validation scores
    if args.show_individual_scores:
        for task_name, val_score in zip(args.task_names, val_scores):
            print(f'Validation {task_name} {args.metric} = {val_score:.6f}')
            
    # save model if improved validation score
    if args.minimize_score and avg_val_score < best_score or \
            not args.minimize_score and avg_val_score > best_score:
        best_score, best_epoch = avg_val_score, epoch
        best_model = copy.deepcopy(model)


Loss = 7.0826e-04, PNorm = 43.2558, GNorm = 0.4867, lr_0 = 1.0000e-04
Validation AVG mae = 4.101726
Validation mu mae = 0.459818
Validation alpha mae = 0.906455
Validation homo mae = 0.004575
Validation lumo mae = 0.004463
Validation gap mae = 0.006130
Validation r2 mae = 32.500538
Validation zpve mae = 0.002201
Validation cv mae = 0.413761
Validation u0 mae = 3.723235
Validation u298 mae = 3.693065
Validation h298 mae = 3.752473
Validation g298 mae = 3.753995


Process Process-510:
Process Process-512:
Process Process-511:
Process Process-507:
Process Process-506:
Process Process-509:
Process Process-508:
Process Process-505:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/Applications/anaconda3/lib/python3.7/multiprocessing/process.py", line 300, in _bootstrap
    util._exit_function()
  File "/Applications/anaconda3/lib/python3.7/multip

KeyboardInterrupt: 

Evaluate best model on test set

In [20]:
# ARGS
args.show_individual_scores = True

In [21]:
# print performance of best model
print(f'Best validation {args.metric} = {best_score:.6f} on epoch {best_epoch}')

Best validation mae = 4.198008 on epoch 27


In [22]:
# return test targets
test_targets = test_data.targets()

# compute predictions and scores on test set
test_preds = predict(
    model=best_model,
    data_loader=test_data_loader,
    scaler=scaler
)
test_scores = evaluate_predictions(
    preds=test_preds,
    targets=test_targets,
    num_tasks=args.num_tasks,
    metric_func=metric_func,
    dataset_type=args.dataset_type,
    logger=logger
)

# average test score
avg_test_score = np.nanmean(test_scores)
print(f'Test {args.metric} = {avg_test_score:.6f}')

# individual test scores
if args.show_individual_scores:
    for task_name, test_score in zip(args.task_names, test_scores):
        print(f'Test {task_name} {args.metric} = {test_score:.6f}')


Test mae = 4.280236
Test mu mae = 0.483251
Test alpha mae = 0.932755
Test homo mae = 0.004504
Test lumo mae = 0.004795
Test gap mae = 0.006307
Test r2 mae = 33.991091
Test zpve mae = 0.002149
Test cv mae = 0.415957
Test u0 mae = 3.848183
Test u298 mae = 3.878890
Test h298 mae = 3.902034
Test g298 mae = 3.892922


In [23]:
# compare with means as predictions of each target

test_targets_array = np.array(test_targets)
target_means = np.mean(test_targets_array,0)
MAE = np.mean(abs(test_targets_array - target_means),0)

avg_test_score = np.nanmean(MAE)
print(f'Test {args.metric} = {avg_test_score:.6f}')

for task_name, test_score in zip(args.task_names, MAE):
    print(f'Test {task_name} {args.metric} = {test_score:.6f}')

Test mae = 25.805655
Test mu mae = 1.192260
Test alpha mae = 6.272279
Test homo mae = 0.016396
Test lumo mae = 0.038681
Test gap mae = 0.040090
Test r2 mae = 178.012015
Test zpve mae = 0.025054
Test cv mae = 3.111326
Test u0 mae = 30.239929
Test u298 mae = 30.239719
Test h298 mae = 30.239719
Test g298 mae = 30.240388
